# Mini-lab 4 - Mapping TAZ travel times

In this lab we will play with some data from the Metropolitan Transportation Commission on travel time from one Traffic Analysis Zone (TAZ) to another. We will use a mapping tool called folium to create a graphical representation of travel times throughout the area.


In [1]:
import folium
import pandas as pd
import geopandas as gpd
import numpy as np

from datascience import *

## The datasets
### MTC travel skims
The Metropolitan Transportation Co,mission (MTC) is the regional transportation planning organization for the Bay Area. They host a database with average travel time, cost, and distance from each traffic analysis zone (TAZ) to all other TAZs in the Bay Area. The files have data for driving alone, car pooling, walking to transit, driving to transit, walking, and biking. 

We have pre-processed the data from the morning commute to include only TAZs around San Francisco, Oakland and Berkeley.

More info on the dataset can be found here - http://analytics.mtc.ca.gov/foswiki/Main/SimpleSkims. 
The descriptions of the columns in the data set are shown below:

|column|description|
|---|---|
|origin|Origin transportation analysis zone|
|destination|Destination transportation analysis zone|
|drive alone|Door-to-door time for the drive alone travel mode (i.e. single occupant private automobile)|
|shared ride (2 people)|Door-to-door time for the shared ride 2 travel mode (i.e. double occupant private automobile)|
|shared ride (3 people)|Door-to-door time for the shared ride 3+ travel mode (i.e. three-or-more occupants traveling in a private vehicle)|
|walk|Door-to-door time for walking|
|bike|Door-to-door time for bicycling|
|walk-transit-walk|Door-to-door time for walk to transit to walk paths|
|drive-transit-walk|Door-to-door time for drive to transit to walk paths|
| walk-transit-drive|Door-to-door time for walk to transit to drive paths (returning home on a park-and-ride tour)|


(The raw data with all bay area TAZs can be found at https://mtcdrive.app.box.com/2015-03-116)

### Bay area TAZ geometry data
GeoJSON is a format used for encoding a variety of geographic data structures. We have saved a GeoJSON file with the Traffic Analysis Zone (TAZ) polygons for the TAZs in the San Francisco, Oakland, and Berkeley region. We will use a mapping package called folium to map the TAZs.

### Read the data
First we read in the sf_oak_traveltimes_bymode.csv

In [2]:
# Read the timeskims datafile using pandas tables
travel_time_data = pd.read_csv('sf_oak_traveltimes_bymode.csv')
travel_time_data.head()

,origin,destination,drive alone,shared ride (2 people),shared ride (3 people),walk,bike,walk-transit-walk,drive-transit-walk,walk-transit-drive
0,1,1,5.92,5.92,5.92,2.4,0.60,-999.00,-999.00,-999.00
1,1,2,6.25,6.25,6.25,4.8,1.20,7.20,5.75,3.82
2,1,3,6.70,6.70,6.70,8.8,2.20,9.70,5.75,4.42
3,1,4,6.66,6.66,6.66,8.2,2.05,6.12,4.18,3.76
4,1,5,7.34,7.34,7.34,13.6,3.40,7.46,5.25,4.87


In [3]:
# look at the data as a datascience Table() using the Table.from_df() function
travel_time_data_ds_table = Table.from_df(travel_time_data)

**Question:** what is the drive alone travel time from origin TAZ 10 (in downtown SF) to destination TAZ 
1019 (the TAZ at UC Berkeley) according to this dataset?

Hint: use Pandas or transform a table to a datascience table and use a where() statement to find the answer.

In [4]:
# Your code and anwer here:

# datascience users: use this hint and apply where statement!
travel_time_data_ds_table.where('origin', 10).where('destination', 1019)['drive alone']

array([23.64])

# Part 2: Use the mapping tools to visualise the TAZs data

As mentioned above, geojson is a standard format for encoding geographic data. 'SF_Oak_TAZs.geojson' is a geojson file that contains the TAZ geometry for TAZs in the San Francisco and Oakland area. We can use the mapping tools we have learned to create travel time maps.

In [5]:
taz_geo_path = 'SF_Oak_TAZs.geojson'

# Using maps to convey information


In [6]:
# load geo data and look at the contents
taz_geometries = gpd.read_file(taz_geo_path)
taz_geometries.head()

,County_FIP,UrbanTAZac,TAZ1454,geometry
0,6075,20.074805,1,POLYGON ((-122.3962860018482 37.79333799956961...
1,6075,31.364645,2,POLYGON ((-122.3995499982689 37.79141500004747...
2,6075,14.595786,3,POLYGON ((-122.4040450015672 37.79076700146884...
3,6075,20.571218,4,POLYGON ((-122.3991319972654 37.79110199879706...
4,6075,49.489970,5,POLYGON ((-122.4032569981368 37.78780900006728...


## Color the map based on the 'bike' travel time from one origin TAZ to all other TAZs.
In the next few cells you will create a **very cool** data visualization tool to visualize the travel time from an origin TAZ to all destination TAZs.

In [7]:
travel_time_data_1 = travel_time_data[travel_time_data['origin']==1][['destination', 'bike']]

In [8]:
bay_area_map = folium.Map(location=[38, -122], zoom_start=7)

In [9]:
bay_area_map.choropleth(
    geo_path=taz_geo_path, # path to the geojson file with state geometries
    data=travel_time_data_1, 
    columns=['destination', 'bike'],
    key_on='properties.TAZ1454', # IMPORTANT! use 'properties'
    fill_color='YlGn',
    threshold_scale=[5, 10, 15, 20, 25, 30]
)

In [15]:
bay_area_map.save(outfile='bay_area_cycling_from_1.html')

# Color unreachale TAZs Grey (optional)
For the datasets provided, MTC follows a convention of setting travel time = '-999' in the data table if they have determined that a mode is not feasible to get from an origin TAZ to a destination TAZ. For example, a bike trip from San Francisco to Oakland is not possible because bikes are not allowed accross the Bay Bridge. Similarly some walk distances are determined to be too far to make a trip on foot.
Color all TAZs with a travel time of -999 grey. Hint - use a procedure similar to the procedure used to color the origin blue.

# Put it all in one method (optional)

Create a function that produces a travel time map. You have all of the components above, just fill them in to the function below.

In [16]:
def create_travel_time_map(mtc_data,
                           taz_geo_path='SF_Oak_TAZs.geojson',
                           origin_taz_id=1,
                           travel_mode='drive alone',
                           color_palette='YlOrRd', 
                           threshold_scale=[5,10,15,20,25]):
    """ 
    Args:
        mtc_data (pd.DataFrame): travel time data between origin and destiantion TAZs, by mode
        taz_geo_path (str): path to geo file with TAZ geometries (for example 'SF_Oak_TAZs.geojson')
        origin_taz_id (int): id of the origin of travel
        travel_mode (str): one of the trael modes in mtc_data ('drive alone', 'bike', etc)
        color_palette (str): folium color palette codename 
        threshold_scale (list): list of thesholds of the choropleth colors
    
    Returns: (folium.Map) a folium map with an overlay of travel times by mode from an origin
    """
    travel_time_data = mtc_data[mtc_data['origin']==origin_taz_id][['destination', travel_mode]]
    
    bay_area_travel_time_map = folium.Map(location=[38, -122], zoom_start=10)
    
    bay_area_travel_time_map.choropleth(
        geo_path=taz_geo_path, # path to the geojson file with state geometries
        data=travel_time_data, 
        columns=['destination', travel_mode],
        key_on='properties.TAZ1454', # IMPORTANT! use 'properties'
        fill_color='YlGn',
        threshold_scale=[5, 10, 15, 20, 25, 30]
    )
    
    return bay_area_travel_time_map

Test your method 

In [17]:
# create a map using desired arguments
my_map = create_travel_time_map(travel_time_data, origin_taz_id = 1019)

# save it as a file
my_map.save(outfile='my_bay_area_map.html')

# Explore travel times for other modes of transportation:

In [13]:
# 

# The relationship between spatial proximity and travel time
For each of the following modes: drive, walk to transit, drive to transit, bike, and walk, do you think a straightline distance from one TAZ to another is a good indicator of travel time? Explain why or why not.



In [14]:
# Your answer here:

